In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [3]:
df=pd.read_csv('train.csv',usecols=['Age','Fare','Survived'])

In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(714, 3)

In [6]:
df.head()

,Survived,Age,Fare
0,0,22.0,7.2500
1,1,38.0,71.2833
2,1,26.0,7.9250
3,1,35.0,53.1000
4,0,35.0,8.0500


In [7]:
x=df.iloc[:,1:]
y=df.iloc[:,0]

In [8]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [9]:
X_train.head(2)

,Age,Fare
328,31.0,20.5250
73,26.0,14.4542


In [10]:
clf=DecisionTreeClassifier()

In [11]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [12]:
accuracy_score(y_test,y_pred)

0.6363636363636364

In [13]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.624706572769953

In [30]:
kbin_age=KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')
kbin_fare=KBinsDiscretizer(n_bins=15,encode='ordinal',strategy='quantile')

In [31]:
trf=ColumnTransformer([
   ('first',kbin_age,[0]),
   ('second',kbin_fare,[1])
    
])

In [32]:
X_train_trf=trf.fit_transform(X_train)
X_test_trf=trf.transform(X_test)

In [33]:
trf.named_transformers_

{'first': KBinsDiscretizer(encode='ordinal', n_bins=15),
 'second': KBinsDiscretizer(encode='ordinal', n_bins=15)}

In [34]:
trf.named_transformers_['first']

KBinsDiscretizer(encode='ordinal', n_bins=15)

In [35]:
trf.named_transformers_['first'].n_bins_

array([15])

In [36]:
trf.named_transformers_['first'].bin_edges_

array([array([ 0.42,  6.  , 16.  , 19.  , 21.  , 23.  , 25.  , 28.  , 30.  ,
              32.  , 35.  , 38.  , 42.  , 47.  , 54.  , 80.  ])             ],
      dtype=object)

In [37]:
output=pd.DataFrame({
    'age':X_train['Age'],
    'age_trf':X_train_trf[:,0],
    'fare':X_train['Fare'],
    'fare_trf':X_train_trf[:,1]
})

In [38]:
output['age_labels']=pd.cut(x=X_train['Age'],
                           bins=trf.named_transformers_['first'].bin_edges_[0].tolist())
output['fare_labels']=pd.cut(x=X_train['Fare'],
                           bins=trf.named_transformers_['second'].bin_edges_[0].tolist())


In [39]:
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
432,42.0,12.0,26.0000,9.0,"(38.0, 42.0]","(18.75, 26.0]"
373,22.0,4.0,135.6333,14.0,"(21.0, 23.0]","(108.9, 512.329]"
302,19.0,3.0,0.0000,0.0,"(16.0, 19.0]",NaN
864,24.0,5.0,13.0000,6.0,"(23.0, 25.0]","(10.5, 13.0]"
480,9.0,1.0,46.9000,11.0,"(6.0, 16.0]","(31.275, 51.479]"


In [40]:
clf=DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)
y_pred2=clf.predict(X_test_trf)

In [41]:
accuracy_score(y_test,y_pred)

0.6363636363636364

In [47]:
X_trf=trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),X_trf,y,cv=10,scoring='accuracy'))

0.6723004694835681